In [ ]:
!pip3 install transformers seqeval 
!pip3 install torch torchvision torchaudio
!pip3 install accelerate
!pip3 install datasets ipywidgets

In [11]:
import transformers

print(transformers.__version__)

4.39.2


In [12]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

# Fine-tuning a model on a token classification task

In [13]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "roberta-base"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [14]:
from datasets import load_dataset, load_metric

In [15]:
# datasets = load_dataset("conll2003")
# datasets = load_dataset("conll2012_ontonotesv5", "english_v12")

datasets = load_dataset("./englishv12/")

Generating train split:   0%|          | 0/115812 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15680 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12217 [00:00<?, ? examples/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 115812
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 15680
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 12217
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [17]:
# datasets["train"][0]["sentences"]

# p = [ {"tokens": sentence["words"], "ner_tags": sentence["named_entities"]} for docu in datasets["train"] for sentence in docu["sentences"]]
# len(p)
# for index, _ in enumerate(p):
#     p[index]["id"] = index
# p[2232]
# datasets["train"].features["sentences"][0]["named_entities"].feature.names

In [18]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PERSON', 'I-PERSON', 'B-NORP', 'I-NORP', 'B-FAC', 'I-FAC', 'B-ORG', 'I-ORG', 'B-GPE', 'I-GPE', 'B-LOC', 'I-LOC', 'B-PRODUCT', 'I-PRODUCT', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME', 'B-PERCENT', 'I-PERCENT', 'B-MONEY', 'I-MONEY', 'B-QUANTITY', 'I-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'B-CARDINAL', 'I-CARDINAL', 'B-EVENT', 'I-EVENT', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-LAW', 'I-LAW', 'B-LANGUAGE', 'I-LANGUAGE'], id=None), length=-1, id=None)

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [19]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
# label_list = datasets["train"].features["sentences"][0]["named_entities"].feature.names
label_list

['O',
 'B-PERSON',
 'I-PERSON',
 'B-NORP',
 'I-NORP',
 'B-FAC',
 'I-FAC',
 'B-ORG',
 'I-ORG',
 'B-GPE',
 'I-GPE',
 'B-LOC',
 'I-LOC',
 'B-PRODUCT',
 'I-PRODUCT',
 'B-DATE',
 'I-DATE',
 'B-TIME',
 'I-TIME',
 'B-PERCENT',
 'I-PERCENT',
 'B-MONEY',
 'I-MONEY',
 'B-QUANTITY',
 'I-QUANTITY',
 'B-ORDINAL',
 'I-ORDINAL',
 'B-CARDINAL',
 'I-CARDINAL',
 'B-EVENT',
 'I-EVENT',
 'B-WORK_OF_ART',
 'I-WORK_OF_ART',
 'B-LAW',
 'I-LAW',
 'B-LANGUAGE',
 'I-LANGUAGE']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [20]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    print(dataset.features.items())
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [21]:
show_random_elements(datasets["train"])

dict_items([('id', Value(dtype='string', id=None)), ('tokens', Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)), ('ner_tags', Sequence(feature=ClassLabel(names=['O', 'B-PERSON', 'I-PERSON', 'B-NORP', 'I-NORP', 'B-FAC', 'I-FAC', 'B-ORG', 'I-ORG', 'B-GPE', 'I-GPE', 'B-LOC', 'I-LOC', 'B-PRODUCT', 'I-PRODUCT', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME', 'B-PERCENT', 'I-PERCENT', 'B-MONEY', 'I-MONEY', 'B-QUANTITY', 'I-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'B-CARDINAL', 'I-CARDINAL', 'B-EVENT', 'I-EVENT', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-LAW', 'I-LAW', 'B-LANGUAGE', 'I-LANGUAGE'], id=None), length=-1, id=None))])


,id,tokens,ner_tags
0,None,"[He, spreads, them, everywhere, possible, until, they, are, hanging, from, the, bridges, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,None,"[There, was, also, the, matter, of, performing, the, prescribed, observances, to, heaven, and, earth, ,, the, ancestors, ,, the, kitchen, god, and, various, other, deities, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,None,"[Individuals, eager, for, tax, -, free, income, drove, up, bond, prices, ,, making, state, and, local, government, debt, one, of, the, best, -, performing, types, of, fixed, -, income, investments, during, the, period, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,None,"[And, what, is, most, shocking, is, the, birth, of, "", super, bacteria, "", that, feed, on, antibiotics, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,None,"[``, I, happen, to, think, it, was, right, ,, '', Mr., Bush, said, flatly, .]","[O, O, O, O, O, O, O, O, O, O, O, B-PERSON, O, O, O]"
5,None,"[The, company, earned, $, 6.6, million, in, the, year, -, earlier, quarter, .]","[O, O, O, B-MONEY, I-MONEY, I-MONEY, O, B-DATE, I-DATE, I-DATE, I-DATE, I-DATE, O]"
6,None,"[Many, overseas, Chinese, and, community, groups, gathered, there, to, wait, for, the, commencement, of, the, Chinese, Culture, and, Costume, Parade, .]","[O, O, B-NORP, O, O, O, O, O, O, O, O, O, O, O, B-EVENT, I-EVENT, I-EVENT, I-EVENT, I-EVENT, I-EVENT, O]"
7,None,"[mhm, .]","[O, O]"
8,None,"[I, know, someone, who, used, 10W40, and, it, gummed, up, her, engine, and, ruined, it., &, quot, ;]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,None,"[It, was, a, huge, fear, of, mine, that, it, would, n't, work, and, that, I, 'd, have, to, be, a, pin, cushion, and, try, this, one, /.]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [38]:
from transformers import AutoTokenizer

# add add_prefix_space=True for Roberta
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [39]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [40]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [0, 20920, 6, 42, 16, 65, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [41]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [0, 20920, 2156, 42, 16, 65, 3645, 3462, 88, 1617, 479, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [42]:
example = datasets["train"][4]
print(example["tokens"])

['It', 'is', 'composed', 'of', 'a', 'primary', 'stele', ',', 'secondary', 'steles', ',', 'a', 'huge', 'round', 'sculpture', 'and', 'beacon', 'tower', ',', 'and', 'the', 'Great', 'Wall', ',', 'among', 'other', 'things', '.']


In [43]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', 'ĠIt', 'Ġis', 'Ġcomposed', 'Ġof', 'Ġa', 'Ġprimary', 'Ġste', 'le', 'Ġ,', 'Ġsecondary', 'Ġst', 'el', 'es', 'Ġ,', 'Ġa', 'Ġhuge', 'Ġround', 'Ġsculpture', 'Ġand', 'Ġbeacon', 'Ġtower', 'Ġ,', 'Ġand', 'Ġthe', 'ĠGreat', 'ĠWall', 'Ġ,', 'Ġamong', 'Ġother', 'Ġthings', 'Ġ.', '</s>']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [44]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(28, 33)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [45]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 9, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [46]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

33 33


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [47]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [48]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [49]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[0, 653, 761, 9, 3783, 17487, 2], [0, 166, 32928, 9603, 47, 7, 1183, 10, 780, 5403, 9, 15581, 436, 479, 2], [0, 15584, 3082, 3192, 23959, 15, 5, 2860, 3875, 9, 436, 4832, 41876, 38628, 9, 15643, 24610, 4743, 2], [0, 15125, 6764, 15, 15643, 24610, 4743, 16, 5, 23001, 7, 5, 41184, 6304, 25132, 23909, 479, 2], [0, 85, 16, 14092, 9, 10, 2270, 11235, 459, 2156, 5929, 1690, 523, 293, 2156, 10, 1307, 1062, 18185, 8, 30943, 9368, 2156, 8, 5, 2860, 2298, 2156, 566, 97, 383, 479, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, -100], [-100, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, -100], [-100, 0, 0, 0, 11, 11, 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [50]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/115812 [00:00<?, ? examples/s]

Map:   0%|          | 0/15680 [00:00<?, ? examples/s]

Map:   0%|          | 0/12217 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [51]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [52]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [53]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [54]:
metric = load_metric("seqeval")

/tmp/ipykernel_4235/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/home/szzonly/.conda/envs/pytorch2/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


This metric takes list of labels for the predictions and references:

In [55]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'WORK_OF_ART': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [56]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [57]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/szzonly/.conda/envs/pytorch2/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


We can now finetune our model by just calling the `train` method:

In [58]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.085100,0.087244,0.844410,0.905512,0.873895,0.974433
2,0.057500,0.078487,0.866885,0.903296,0.884716,0.976629
3,0.039300,0.083715,0.877960,0.899919,0.888804,0.977322


TrainOutput(global_step=21717, training_loss=0.07199648306306874, metrics={'train_runtime': 3111.6637, 'train_samples_per_second': 111.656, 'train_steps_per_second': 6.979, 'total_flos': 9837948416228664.0, 'train_loss': 0.07199648306306874, 'epoch': 3.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [59]:
trainer.evaluate()

{'eval_loss': 0.083714559674263,
 'eval_precision': 0.8779600521655571,
 'eval_recall': 0.8999190910050304,
 'eval_f1': 0.8888039607400331,
 'eval_accuracy': 0.9773220862258488,
 'eval_runtime': 35.1039,
 'eval_samples_per_second': 446.673,
 'eval_steps_per_second': 27.917,
 'epoch': 3.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [60]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CARDINAL': {'precision': 0.8178557194101236,
  'recall': 0.9023746701846965,
  'f1': 0.8580388877273677,
  'number': 2274},
 'DATE': {'precision': 0.8491158014831717,
  'recall': 0.8792085056113408,
  'f1': 0.8639001741149158,
  'number': 3386},
 'EVENT': {'precision': 0.6157894736842106,
  'recall': 0.4936708860759494,
  'f1': 0.5480093676814989,
  'number': 237},
 'FAC': {'precision': 0.4878048780487805,
  'recall': 0.6741573033707865,
  'f1': 0.5660377358490566,
  'number': 178},
 'GPE': {'precision': 0.9328788673308862,
  'recall': 0.9417681312863949,
  'f1': 0.9373024236037935,
  'number': 5667},
 'LANGUAGE': {'precision': 0.9696969696969697,
  'recall': 0.8205128205128205,
  'f1': 0.8888888888888888,
  'number': 39},
 'LAW': {'precision': 0.5647058823529412,
  'recall': 0.6575342465753424,
  'f1': 0.6075949367088608,
  'number': 73},
 'LOC': {'precision': 0.7694174757281553,
  'recall': 0.7389277389277389,
  'f1': 0.7538644470868014,
  'number': 429},
 'MONEY': {'precision': 0.